<h1>Stair landing

<h2>[Reference project model](http://stairs4u.com/code/stair_landing_minimum_widths.htm)</h2>
![alt text](model_reference.png)

<h4>Variables:</h4>  <b>alzata</b>: ideal riser, 
            <b>gradini</b>: stairs's number (maybe decimal number), 
            <b>resto</b>: eventual decimal of 'gradini', 
            <b>totGradiniIntero</b>: integer part of 'gradin' (the effettive stairs's number), 
            <b>newAlzata</b>: effettive riser, 
            <b>pedata</b>: ideal tread, 
            <b>yLanding</b>: landing's depth, 
            <b>pedataFraz</b>: sum treads stairs without landing's depth, 
            <b>newPedata</b>: effettive tread of each stair, 
            <b>larghezzaStair</b>: stair's width.

<h4>Description of the geometric method i'm going to implement:</h4> this feature allows you to create stairs and landings from third formal parameters passed to it. With the parameters passed, i will define the numbers of the steps with their tread and riser and the size of the landings.
The numbers of the steps according to the formal parameters can be decimal. To resolve this issue before I go on to take the integer part of the result of the division, and the rest of the division is proportionally added to each stair.
To create the stairs and landing of the first flight use a for that step will create the single stair according to the formal parameters, through the use of CUBOID. This starts at 1 for up to the total number of steps previously obtained.
For the second ramp simply I do the same operation as described before, and then go to assemble all via STRUCT, after appropriate rotation of 180 ° and translating the second ramp.
The translation will be made on all 3 axes, the x-axis for the accuracy of a factor dx; the y-axis by a factor equal to the product between the tread and the number of steps of the single ramp, net of a tread; on the Z axis by a factor equal to the product between the riser and the number of the single ramp steps.

Import libraries

In [28]:
from pyplasm import *
from larlib import *

Define principal function with 3 formal parameters, named dx,dy,dz

In [29]:
#2alzata+pedata=64cm ideal formule of stair's size
def ggpl_stair_landing(dx,dy,dz):
    
    #variables
    alzata = .2
    gradini = dz/alzata    
    resto = modf(gradini)[0]    
    totGradiniIntero = modf(gradini)[1]
    newAlzata = dz/totGradiniIntero
    pedata =.64-(2*newAlzata)
    yLanding = dy/4.0
    pedataFraz = (dy-yLanding)
    newPedata = pedataFraz/(totGradiniIntero)
    larghezzaStair = dx/2.0
    
    #construct landing as a cuboid with the size calculate before
    landing = CUBOID([larghezzaStair*2,yLanding,newAlzata])
    
    #contruct first stair landing
    #i create every each single stair until the total stairs's numer. 
    #stair is create as a cuboid with the size calculate before
    stairsLanding=[]
    for i in range (1,int(totGradiniIntero/2)):
        stairsLanding.append(CUBOID([larghezzaStair,newPedata,newAlzata]))
        stairsLanding.append(T([2,3])([newPedata,newAlzata]))
    stairsLanding.append(landing)
    
    #construct second stair landing
    #i create every each single stair until the total stairs's number. 
    #stair is create as a cuboid with the size calculate before
    stairsLanding2=[]
    for j in range (1,int(totGradiniIntero/2)):
        stairsLanding2.append(CUBOID([larghezzaStair,newPedata,newAlzata]))
        stairsLanding2.append(T([2,3])([newPedata,newAlzata]))
    stairsLanding2.append(landing)
    
    #contruct the total of stair landing, as a struct of stair landing.
    #Before the construct, i rotate the second stair landing of 180°.
    #At the end i traslate the second stair landing
    firstStairsLanding = STRUCT(stairsLanding)
    secondStairsLandingRotate = STRUCT([R([1,2])(PI),STRUCT(stairsLanding2)])
    secondStairsLanding = STRUCT([T([1,2,3])([dx,(newPedata*((totGradiniIntero)/2)-newPedata),(newAlzata*((totGradiniIntero)/2))]),secondStairsLandingRotate])
    VIEW(STRUCT([firstStairsLanding,secondStairsLanding]))

Launch function

In [30]:
ggpl_stair_landing(1.94,5,3.2)

<b>Intermediate steps</b>

<b>stairsLanding: just one ramp</b>
![alt text](firstStairLanding.jpg)

<b>secondStairsLandingRotate: 2 ramps without any traslation of the second's</b>
![alt text](noTraslation.jpg)

<h2>Final result with different sizes!</h2>

<b>stairsLanding: 2 semi complete ramps. Sizes: dx=1.94, dy=5,dz=3.2</b>
![alt text](semiFinal.jpg)

<b>stairsLanding: 2 semi complete ramps. Sizes: dx=3, dy=8,dz=5.2</b>
![alt text](semiFinal2.jpg)

<b>stairsLanding: 2 semi complete ramps. Sizes: dx=4, dy=18,dz=8</b>
![alt text](semiFinal3.jpg)

<b>stairsLanding: 2 semi complete ramps. Sizes: dx=1, dy=3,dz=2</b>
![alt text](semiFinal4.jpg)